In [10]:
import torch
import torch.nn as nn
from torch.distributions import normal
from torch.distributions import uniform
import math

In [34]:
class CUDAvoir(nn.Module):
    def __init__(self,n,p,rho,o=0.0,bias=True):
        super(CUDAvoir,self).__init__()

        self.n = torch.tensor(n)
        self.p = torch.tensor(p)
        self.rho = torch.tensor(rho)

        self.v = torch.zeros(self.n) ## Recurrent Layer State Vector
        self.w = torch.zeros(self.n,self.n) ## Recurrent Layer Weight Matrix

        self.ol = nn.Linear(self.n, 1, bias=False) ## Linear Output Layer
        self.o = torch.tensor([o]) ## Initalize Output Neuron
        self.fb = nn.Linear(1, self.n, bias=False) ## Linear Feedback Layer
        self.fb.weight.requires_grad_(False)

        if bias: ## Recurrent Layer Bias
            self.b = torch.FloatTensor(n).uniform_(0,1)
        else:
            self.b = torch.zeros(self.n)
        
        ## Populate Recurrent Layer Weight Matrix
        norm = normal.Normal(loc=0,scale=self.rho)
        uni = uniform.Uniform(0,1) 
        for i in range(self.n):
            for j in range(self.n):
                uni_draw = uni.sample()
                if uni_draw < self.p:
                    self.w[i,j] = norm.sample()
    
    def forward(self):
        z = torch.matmul(self.w,self.v) + self.fb(self.o) + self.b
        nl = nn.Tanh()
        y = nl(z)
        self.v = y
        self.o = self.ol(y)
        return self.o

In [38]:
n = 500
p = .01
rho = 0.5
m = CUDAvoir(n=n,p=p,rho=rho,o=0.0,bias=True)

In [37]:
errs = []
lr = 0.02
steps = 500
dt = 0.05
targ = torch.sin
for s in range(2):
    o = m.forward()
    print(o)
    u = torch.tensor([dt*s])
    for i in range(100):
        err = o - targ(u)
        errs.append(err)
        errsq = err.pow(2)
        errsq.backward(retain_graph=True)
        ol = m.ol.weight.detach()
        ol_g = m.ol.weight.grad
        nw = (ol - lr*ol_g).clone().detach()
        m.ol.weight = nn.Parameter(nw)
        m.o = m.ol(m.v)
    print(errs)
    errs = []

tensor([nan], grad_fn=<SqueezeBackward3>)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'